In [1]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 814 kB/s eta 0:00:01


In [1]:
import gc
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
import numpy as np
import lightgbm as lgb
import time

训练集

In [2]:
user = pd.read_csv('data/train_preliminary/user.csv')

In [3]:
ad = pd.read_csv('output/features/train/ad_id_embedding_df.csv')
agg = pd.read_csv('output/features/train/aggregate_features_df.csv')
creative = pd.read_csv('output/features/train/creative_id_embedding_df.csv')
mode = pd.read_csv('output/features/train/mode_features_df.csv')

In [4]:
train = pd.merge(user, ad, how='outer')
del user, ad
gc.collect()
train = pd.merge(train, agg, how='outer')
del agg
gc.collect()
train = pd.merge(train, creative, how='outer')
del creative
gc.collect()
train = pd.merge(train, mode, how='outer')
del mode
gc.collect()
train.shape

(900000, 213)

测试集

In [5]:
ad = pd.read_csv('output/features/test/ad_id_embedding_df.csv')
agg = pd.read_csv('output/features/test/aggregate_features_df.csv')
creative = pd.read_csv('output/features/test/creative_id_embedding_df.csv')
mode = pd.read_csv('output/features/test/mode_features_df.csv')


test = pd.merge(ad, agg, how='outer')
del agg
gc.collect()
test = pd.merge(test, creative, how='outer')
del creative
gc.collect()
test = pd.merge(test, mode, how='outer')
del mode
gc.collect()
test.shape

(1000000, 211)

In [6]:
features = [ x for x in train.columns if x not in ['user_id', 'age', 'gender']]
len(features)

210

In [7]:
target1 = train['age'] - 1
target2 = train['gender'] - 1

In [8]:
params1 = {
#     'max_depth':14,
#     'num_leaves': 64,
#     'min_data_in_leaf': 20,
#     'min_child_samples':20,
    'objective': 'multiclassova',
    'num_class': 10,
    'learning_rate': 0.1,
    "boosting": "gbdt",
#         'max_bin': 255,
#     'bagging_fraction': 0.85,
#     "feature_fraction": 0.8,
#     "bagging_freq": 1,
#     "bagging_fraction": 0.85,
#     "bagging_seed": 23,
#     "metric": 'logloss',
#     "lambda_l1": 0.2,
#     "nthread": 4,
}

In [9]:
params2 = {
    'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.11
}

In [10]:
def lgb_model_2(n_splits, random_state, train, test, features, target, params):
    start = time.clock()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    #验证集
    oof_lgb = np.zeros(len(train))
    #测试集
    predictions_lgb = np.zeros(len(test))

    for fold_, (train_idx, valid_idx) in enumerate(folds.split(train[features].values, target.values)):
        print('fold：{} start...'.format(fold_))
        #训练集和验证集的划分
        train_data = lgb.Dataset(train[features].iloc[train_idx], label=target.iloc[train_idx])
        valid_data = lgb.Dataset(train[features].iloc[valid_idx], label=target.iloc[valid_idx])
        
        clf = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data],
            verbose_eval = 100,
            early_stopping_rounds=100 
        )
        
        del train_data, valid_data
        gc.collect()
        
        #对验证集进行预测
        val_pred = clf.predict(train[features].iloc[valid_idx], num_iteration=clf.best_iteration) # 每折 验证集的概率
        oof_lgb[valid_idx] = val_pred                    #整个  验证集的概率
        val_target = target2.iloc[valid_idx]             #验证集的真实标签
        val_pred =(val_pred>=0.499).astype('int')        #验证集的预测标签
        print("fold_{}：valid_acc：{}".format(fold_, accuracy_score(val_target, val_pred)))   #每折  验证集的ACC
        
        #对测试集进行预测
        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
        print('fold：{} end...\n'.format(fold_))
    
    predictions_lgb = (predictions_lgb>=0.499).astype('int')
    
    print('评分信息')
    print("ROC: {:<8.5f}".format(roc_auc_score(target2, oof_lgb)))
    valid_pre = (oof_lgb>=0.499).astype('int')
    gender_acc = accuracy_score(target2,valid_pre)
    print("accuracy: {:<8.5f}".format(gender_acc))
    
    print("run time：{}".format((time.clock() - start) / 120))
    return valid_pre, predictions_lgb

In [17]:
def lgb_model_10(n_splits, random_state, train, test, features, target, params):
    start = time.clock()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    #验证集
    oof_lgb = np.zeros((len(train), 10))
    #测试集
    predictions_lgb = np.zeros((len(test), 10))

    for fold_, (train_idx, valid_idx) in enumerate(folds.split(train[features].values, target.values)):
        print('fold：{} start...'.format(fold_))
        #训练集和验证集的划分
        train_data = lgb.Dataset(train[features].iloc[train_idx], label=target.iloc[train_idx])
        valid_data = lgb.Dataset(train[features].iloc[valid_idx], label=target.iloc[valid_idx])
        
        clf = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data],
            verbose_eval = 1000,
            early_stopping_rounds=100 
        )
        
        del train_data, valid_data
        gc.collect()
        
        #对验证集进行预测
        val_pred = clf.predict(train[features].iloc[valid_idx], num_iteration=clf.best_iteration)
        oof_lgb[valid_idx] = val_pred
        val_target = target2.iloc[valid_idx]
        val_pred = np.argmax(val_pred, axis=1)
        print("fold_{}：valid_F1：{}".format(fold_, f1_score(val_pred, val_target, average='macro')))
        
        #对测试集进行预测
        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
        print('fold：{} end...\n'.format(fold_))
    
    predictions_lgb = np.argmax(predictions_lgb, axis=1)
    
    print('评分信息')
    valid_pre = np.argmax(oof_lgb, axis=1)    
    print("F1:",f1_score(valid_pre,target,average='macro'))

    print("run time：{}".format(time)
    return valid_pre, predictions_lgb


gender

In [19]:
valid_pre_gender, pre_gender = lgb_model_2(5, 2020, train, test, features, target2, params2)

fold：0 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965747	valid_1's auc: 0.964952
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.965747	valid_1's auc: 0.964952
fold_0：valid_acc：0.9147666666666666
fold：0 end...

fold：1 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.966029	valid_1's auc: 0.963291
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.966029	valid_1's auc: 0.963291
fold_1：valid_acc：0.9140277777777778
fold：1 end...

fold：2 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965892	valid_1's auc: 0.964333
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.965892	valid_1's auc: 0.964333
fold_2：valid_acc：0.9148555555555555
fold：2 end...

fold：3 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.965988	valid_1's auc: 0.9635
Did not 

In [20]:
test['predicted_age'] = -1

In [21]:
test['predicted_gender'] = pre_gender + 1

In [22]:
test

,user_id,ad_id_embe_0_mean,ad_id_embe_0_min,ad_id_embe_0_max,ad_id_embe_1_mean,ad_id_embe_1_min,ad_id_embe_1_max,ad_id_embe_2_mean,ad_id_embe_2_min,ad_id_embe_2_max,...,creative_id_embe_31_min,creative_id_embe_31_max,creatice_id_mode,ad_id_mode,product_id_mode,product_category_mode,advertiser_id_mode,industry_mode,predicted_age,predicted_gender
0,3153317,-2.682523,-8.669187,2.636314,-0.667143,-3.956190,5.552232,0.507184,-4.488916,7.713791,...,-1.955285,5.185283,142315,127841,0,7,1134,0,-1,1
1,3284714,1.311700,-6.961670,13.033854,-2.198414,-6.055876,1.041636,0.800665,-1.691322,4.737873,...,-0.630476,1.305794,1,1,0,18,2623,6,-1,2
2,3491929,-1.134799,-5.005258,8.467810,-1.922497,-5.327207,1.509199,0.864347,-1.864425,6.604331,...,-1.990977,3.632098,1,1,0,5,4117,0,-1,1
3,3831238,0.697868,-4.388091,6.418326,0.967878,-1.172875,9.640598,1.634856,-2.271515,11.674409,...,-0.690808,6.479375,1,1,0,3,25732,247,-1,1
4,3304452,-2.374031,-16.744963,12.799192,-1.743596,-5.674638,2.011278,0.001953,-8.734511,5.774693,...,-2.544380,5.582256,2639577,2271868,0,18,14681,0,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,3274620,-0.917764,-2.590725,0.866687,-4.247491,-9.327357,-1.060171,-1.807481,-4.037711,1.559009,...,-0.026618,5.940140,4145994,3556891,0,18,14682,88,-1,1
999996,3818829,-2.500827,-4.954204,-1.072990,0.032536,-0.483146,0.889291,-0.633239,-1.302930,-0.010010,...,0.181182,1.236323,3861244,3317841,0,18,14682,88,-1,1
999997,3380122,3.249248,-0.037565,6.833143,-2.539434,-4.424579,-0.278861,0.361960,-0.934582,3.796174,...,-0.907104,0.434290,4026829,3457126,0,18,57948,24,-1,2
999998,3996272,0.070657,-1.335933,1.381528,-1.093962,-1.895860,-0.685235,-0.789746,-1.928318,-0.167945,...,0.292214,3.097815,4050535,3476774,0,18,9058,6,-1,2


In [23]:
sub = test[['user_id', 'predicted_age', 'predicted_gender']]

In [24]:
sub

,user_id,predicted_age,predicted_gender
0,3153317,-1,1
1,3284714,-1,2
2,3491929,-1,1
3,3831238,-1,1
4,3304452,-1,1
...,...,...,...
999995,3274620,-1,1
999996,3818829,-1,1
999997,3380122,-1,2
999998,3996272,-1,2


In [25]:
sub.to_csv("sub/info+ad+creative_gender.csv",index=False,sep=',')

age

In [18]:
valid_pre_age, pre_age = lgb_model_10(5, 2020, train, test, features, target1, params1)

fold：0 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.53095	valid_1's multi_logloss: 1.58663
fold_0：valid_F1：0.02428759945048469
fold：0 end...

fold：1 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.53031	valid_1's multi_logloss: 1.58688
fold_1：valid_F1：0.024203903915530848
fold：1 end...

fold：2 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.53008	valid_1's multi_logloss: 1.58608
fold_2：valid_F1：0.02401302326717176
fold：2 end...

fold：3 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.53037	valid_1's multi_logloss: 1.58884
fold_3：valid_F1：0.0241782316435745
fold：3 end...



In [19]:
test['predicted_gender'] = -1
test['predicted_age'] = pre_age + 1
sub = test[['user_id', 'predicted_age', 'predicted_gender']]
sub

,user_id,predicted_age,predicted_gender
0,3153317,3,-1
1,3284714,3,-1
2,3491929,3,-1
3,3831238,3,-1
4,3304452,3,-1
...,...,...,...
999995,3274620,6,-1
999996,3818829,3,-1
999997,3380122,6,-1
999998,3996272,3,-1


In [ ]:
sub.to_csv("sub/5+info+ad+creative_age.csv",index=False,sep=',')